Algoritmo de limpieza de la base de datos, para llegar al resultado deseado.

In [ ]:
import pandas as pd
df=pd.read_csv('/content/Datos meli.csv',sep=';')
df['item_id'] = df['item_id'].fillna(0)
df = df[df['event_name'] != 'SEARCH']
df = df.sort_values(['user_id','timestamp','item_id'], ascending=[False,False,False])
# Eliminar filas consecutivas con valores duplicados en col1, col2 y col3
df['is_duplicate'] = (
    (df['event_name'] == df['event_name'].shift(-1)) &
    (df['item_id'] == df['item_id'].shift(-1)) &
    (df['user_id'] == df['user_id'].shift(-1)) &
    (df['event_name'] != 'BUY'))

# Paso 2: Identificar los índices a eliminar (solo la fila con el índice mayor)
indices_to_remove = df[df['is_duplicate']].index + 1

# Paso 3: Filtrar el DataFrame para eliminar las filas con los índices mayores
df = df[~df.index.isin(indices_to_remove)]

# Paso 4: Eliminar la columna auxiliar
df = df.drop(columns='is_duplicate')

mapeo = {
    'SEARCH': 0,
    'PRODUCT': 1,
    'CHECKOUT_1': 2,
    'CHECKOUT_2': 3,
    'CHECKOUT_3': 4,
    'BUY': 5}

# Aplicar el mapeo a la columna y renombrarla a event_num
df['event_num'] = df['event_name'].map(mapeo)
df = df.reset_index(drop=True)


In [ ]:
def eliminar_filas(df):
    indices_a_eliminar = []
    for i in range(len(df) - 1):
        if df.loc[i + 1, 'event_num'] >= df.loc[i, 'event_num'] and df.loc[i + 1, 'event_num'] != 5 and df.loc[i + 1, 'item_id'] == df.loc[i, 'item_id']and df.loc[i + 1, 'user_id'] == df.loc[i, 'user_id']:
            indices_a_eliminar.append(i + 1)

    if indices_a_eliminar:
        df = df.drop(indices_a_eliminar).reset_index(drop=True)
        df = eliminar_filas(df)  # Llamada recursiva
    return df

df_filtered = eliminar_filas(df)

In [ ]:
df_buy = df[df['event_name'] == 'BUY']
df_filtered['date'] = pd.to_datetime(df_filtered['timestamp']).dt.date
df_buy['date'] = pd.to_datetime(df_buy['timestamp']).dt.date
df_merged = pd.merge(df_filtered, df_buy[['item_id', 'user_id', 'date']], on=['item_id', 'user_id', 'date'], how='inner')

def eliminar_filas(df_merged):
    indices_a_eliminar = []
    for i in range(len(df_merged) - 1):
        if df_merged.loc[i + 1, 'event_num'] >= df_merged.loc[i, 'event_num'] and df_merged.loc[i + 1, 'event_num'] != 5 and df_merged.loc[i + 1, 'item_id'] == df_merged.loc[i, 'item_id']and df_merged.loc[i + 1, 'user_id'] == df_merged.loc[i, 'user_id']:
            indices_a_eliminar.append(i + 1)

    if indices_a_eliminar:
        df_merged = df_merged.drop(indices_a_eliminar).reset_index(drop=True)
        df_merged = eliminar_filas(df_merged)  # Llamada recursiva
    return df_merged

df_merged_final = eliminar_filas(df_merged)
print(df_merged_final)
df_merged_final = df_merged_final.drop_duplicates()
df_merged_final.to_csv('/content/df_merged_final.csv', index = False)

<ipython-input-3-97b2f780852b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_buy['date'] = pd.to_datetime(df_buy['timestamp']).dt.date


      event_name      item_id                     timestamp site  \
0            BUY  356305667.0  2021-08-02T08:35:58.583-0400  MLA   
1     CHECKOUT_3  356305667.0  2021-08-02T08:34:42.956-0400  MLA   
2     CHECKOUT_2  356305667.0  2021-08-02T08:32:56.100-0400  MLA   
3        PRODUCT  356305667.0  2021-08-02T08:32:07.822-0400  MLA   
4            BUY  393529429.0  2021-08-02T11:58:20.201-0400  MLA   
...          ...          ...                           ...  ...   
5038     PRODUCT  789004239.0  2021-08-02T19:45:44.269-0400  MLA   
5039         BUY   79821609.0  2021-08-02T08:54:38.662-0400  MLA   
5040  CHECKOUT_3   79821609.0  2021-08-02T08:54:18.937-0400  MLA   
5041  CHECKOUT_2   79821609.0  2021-08-02T08:53:45.257-0400  MLA   
5042  CHECKOUT_1   79821609.0  2021-08-02T08:53:38.860-0400  MLA   

                                            experiments  user_id  event_num  \
0     {buyingflow/user-track=6796, buyingflow/addres...  9975941          5   
1     {buyingflow/user-tr

In [ ]:
import pandas as pd

# Cargar el DataFrame
df = pd.read_csv('/content/df_merged_final.csv')

# Convertir 'timestamp' a datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Función para concatenar los experimentos
def concatenate_experiments(group):
    return ''.join(group['experiments'])

# Filtrar y ordenar para seleccionar el timestamp más antiguo cuando 'event_num' es 5
df_sorted = df.sort_values(by=['user_id', 'item_id', 'event_num', 'timestamp'])
df_filtered = df_sorted.drop_duplicates(subset=['user_id', 'item_id', 'event_num'], keep='first')

# Agrupar por 'user_id' y 'item_id' y concatenar los experimentos por 'event_num'
df_grouped = df_filtered.groupby(['user_id', 'item_id']).apply(concatenate_experiments).reset_index(name='concatenated_experiments')
df_grouped['count'] = df_grouped.groupby('concatenated_experiments')['concatenated_experiments'].transform('count')
#df_grouped = df_grouped.drop(columns=['user_id','item_id'])

# Guardar el resultado en un archivo CSV
df_grouped.to_csv('/content/df_concatenated_experiments.csv', index=False)



Ejemplo de conteo de ocurrencias por un experimento dado para calcular tamaños de muestras

In [ ]:
import pandas as pd
df = pd.read_csv('/content/Datos meli.csv',sep=";")
df_1 = df[df['experiments']=="{search/remove-ecn-tag=4954, vip/showV2V3BoxMessages=6430, search/tendency-landing-enabled-MLA=6980, pdp/viewItemPageMigrationDesktopReviewsNoTabs=4856, cookiesConsentBanner=DEFAULT, vip/shippingCalculatorMigrationModalExperiment=6551, vip/carousel-v2p-above-the-fold=6786, pdp/viewItemPageMigrationReturns=5208, vip/seller-with-tooltip=4692}"].count()
print(df_1)

event_name     2184
item_id        2184
timestamp      2184
site           2184
experiments    2184
user_id        2184
dtype: int64


SRM test - mediante test chi-cuadrado con una significancia de 0.05

In [ ]:
import scipy.stats as stats

# Define the observed number of users in each group
observed = [894, 163]  # Example for Version A, Version B, Version C, and Version D

# Define the expected number of users assuming an equal split across all groups
total_users = sum(observed)
expected = [total_users / len(observed)] * len(observed)  # Expected split

# Perform the chi-square goodness of fit test
chi2_stat, p_value = stats.chisquare(f_obs=observed, f_exp=expected)

print(f"Chi-square Statistic: {chi2_stat}")
print(f"p-value: {p_value}")

# Interpretation
if p_value < 0.05:
    print("The observed distribution differs significantly from the expected distribution. Possible SRM detected.")
else:
    print("No significant difference from the expected distribution. SRM is unlikely.")



Chi-square Statistic: 207.21076233183857
p-value: 5.577214777607944e-47
The observed distribution differs significantly from the expected distribution. Possible SRM detected.


Test binomial para rechazo de hipotesis nulas (EJEMPLO):

In [ ]:
import scipy.stats as stats

# Number of conversions in Version A and B
conversions_A = 44
conversions_B = 59

# Number of users in Version A and B
users_A = 5512
users_B = 5098

# Conversion rate for Version A (used as the hypothesized probability of success under H0)
p = conversions_A / users_A

# Perform the binomial test for Version B
result = stats.binomtest(conversions_B, n=users_B, p=p, alternative='greater')

print(f"p-value: {result.pvalue}")


p-value: 0.003979976890238631
